In [ ]:
data = 'hin.txt'

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 2979

In [ ]:
input_texts=[]
target_texts=[]

with open(data,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text +'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for i in range(1,10):
  print(input_texts[i], target_texts[i])

Duck! 	झुको!

Duck! 	बतख़!

Help! 	बचाओ!

Jump. 	उछलो.

Jump. 	कूदो.

Jump. 	छलांग.

Hello! 	नमस्ते।

Hello! 	नमस्कार।

Cheers! 	वाह-वाह!



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
input_tk  = Tokenizer()
input_tk.fit_on_texts(input_texts)
target_tk = Tokenizer()
target_tk.fit_on_texts(target_texts)

In [ ]:
print('length of the input words',len(input_tk.word_index))
print('length of the target words',len(target_tk.word_index))

length of the input words 2407
length of the target words 3044


In [ ]:
num_encoder_tokens = len(input_tk.word_index)
num_decoder_tokens = len(target_tk.word_index)
max_encoder_seq_length = max([len(x) for x in input_texts])
max_decoder_seq_length = max([len(x) for x in target_texts])

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
input_seq=[]
target_seq=[]

In [ ]:
def remove_pun(seq):
  for x in range(0,len(seq)-1):
        remove_punctuation_nltk(x)
        # Tokenize the text into words
        words = nltk.word_tokenize(x)

        # Remove punctuation from each word
        words_without_punctuation = [word for word in x if word not in string.punctuation]

        # Join the words back into a sentence
        text = ' '.join(words_without_punctuation)

        input_seq.append(text)

In [ ]:
remove_pun(input_texts)
remove_pun(target_texts)

TypeError: expected string or bytes-like object

In [ ]:
from os import remove
from tensorflow.keras.utils import pad_sequences
input_seq = remove_punctuation_nltk(input_texts)
target_seq = remove_punctuation_nltk(target_texta)
input_sequence = pad_sequences(input_seq,maxlen = max_encoder_seq_length, padding = 'pre')
target_sequence = pad_sequences(target_seq,maslen = max_decoder_seq_length, padding ='pre')

TypeError: expected string or bytes-like object

In [ ]:
print('Number of samples (input):',len(input_texts))
print('Number of samples (target):',len(target_texts))
print('Number of unique input tokens:',num_encoder_tokens)
print('Number of unique output tokens:',num_decoder_tokens)
print("Max sequence length for inputs:",max_encoder_seq_length)
print('Max sequence length for outputs:',max_decoder_seq_length)

In [ ]:
input_token_index = dict(
[(char,i)for i, char in enumerate(input_characters)])
target_token_index = dict(
[(char,i)for i, char in enumerate (target_characters)])

In [ ]:
input_token_index

In [ ]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [ ]:
for i,(input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.
        encoder_input_data[i, t+1: , input_token_index[' ']] = 1.
        for t, char in enumerate(target_text):
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                decoder_input_data[i, t, target_token_index[char]]=1.
            decoder_input_data[i, t+1:, target_token_index[char]]=1
            decoder_target_data[i,t:,target_token_index[' ']]=1.

In [ ]:
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h,state_c]

In [ ]:
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h,state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _, = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation ='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs,decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop',loss ='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data],decoder_target_data,batch_size=batch_size, epochs = epochs, validation_split=0.2)

In [ ]:
decoderd_sentence=[]
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim))
decoder_state_input_c = Input(shape=(latent_dim))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
decoder_output, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_output)
decoder_model = Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)

reverse_input_char_index = dict((i,char)for char, i in input_token_index.items())
reverse_target_char_index = dict((i,char)for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1, num_decoder_tokens))
    target_seq[0,0,target_token_index['\t']]=1

    stop_condition = False
    decoded_sentence= ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq]+states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if(sampled_char == '\n' or len(decoded_sentence)>max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index]=1.

        states_value = [h,c]

    return decoded_sentence

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence', input_texts[seq_index])
    print('Decoded sentence:',decoded_sentence)

In [ ]:
encoder_input_data